In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from rapidfuzz import process 
from tkinter import *
from tkinter import ttk

df = pd.read_csv('laptop_price.csv', encoding='latin-1')


df['Ram'] = df['Ram'].str.replace('GB', '').str.strip()
df['Ram'] = pd.to_numeric(df['Ram'], errors='coerce')


def extract_memory_types(mem_str):
    mem_str = str(mem_str).lower()
    ssd = 0
    hdd = 0
    if 'ssd' in mem_str:
        ssd_part = mem_str.split('ssd')[0]
        ssd = int(''.join(filter(str.isdigit, ssd_part)))
    if 'hdd' in mem_str:
        if '+' in mem_str:
            hdd_part = mem_str.split('+')[-1]
        else:
            hdd_part = mem_str
        hdd = int(''.join(filter(str.isdigit, hdd_part)))
    return pd.Series([ssd, hdd])

df[['SSD', 'HDD']] = df['Memory'].apply(extract_memory_types)

company_product_map = df.groupby('Company')['Product'].unique().to_dict()


comp = ['Apple', 'HP', 'Acer', 'Asus', 'Dell', 'Lenovo', 'Chuwi', 'MSI',
        'Microsoft', 'Toshiba', 'Huawei', 'Xiaomi', 'Vero', 'Razer',
        'Mediacom', 'Samsung', 'Google', 'Fujitsu', 'LG']

res = ['IPS Panel Retina Display 2560x1600', '1440x900',
       'Full HD 1920x1080', 'IPS Panel Retina Display 2880x1800',
       '1366x768', 'IPS Panel Full HD 1920x1080',
       'IPS Panel Retina Display 2304x1440',
       'IPS Panel Full HD / Touchscreen 1920x1080',
       'Full HD / Touchscreen 1920x1080',
       'Touchscreen / Quad HD+ 3200x1800',
       'IPS Panel Touchscreen 1920x1200', 'Touchscreen 2256x1504',
       'Quad HD+ / Touchscreen 3200x1800', 'IPS Panel 1366x768',
       'IPS Panel 4K Ultra HD / Touchscreen 3840x2160',
       'IPS Panel Full HD 2160x1440', '4K Ultra HD / Touchscreen 3840x2160',
       'Touchscreen 2560x1440', '1600x900', 'IPS Panel 4K Ultra HD 3840x2160',
       '4K Ultra HD 3840x2160', 'Touchscreen 1366x768',
       'IPS Panel Full HD 1366x768', 'IPS Panel 2560x1440',
       'IPS Panel Full HD 2560x1440', 'IPS Panel Retina Display 2736x1824',
       'Touchscreen 2400x1600', '2560x1440', 'IPS Panel Quad HD+ 2560x1440',
       'IPS Panel Quad HD+ 3200x1800', 'IPS Panel Quad HD+ / Touchscreen 3200x1800',
       'IPS Panel Touchscreen 1366x768', '1920x1080',
       'IPS Panel Full HD 1920x1200', 'IPS Panel Touchscreen / 4K Ultra HD 3840x2160',
       'IPS Panel Touchscreen 2560x1440', 'Touchscreen / Full HD 1920x1080',
       'Quad HD+ 3200x1800', 'Touchscreen / 4K Ultra HD 3840x2160',
       'IPS Panel Touchscreen 2400x1600']

prod = df['Product'].unique().tolist()
mem = [4, 8, 12, 16, 32]
disk = [0, 64, 128, 256, 512, 1024]


company_encoder = OrdinalEncoder(categories=[comp], dtype=int)
screenres_encoder = OrdinalEncoder(categories=[res], dtype=int)
product_encoder = LabelEncoder()


df['Company_enc'] = company_encoder.fit_transform(df[['Company']])
df['ScreenRes_enc'] = screenres_encoder.fit_transform(df[['ScreenResolution']])
df['Prod_enc'] = product_encoder.fit_transform(df['Product'])

df['Price_in_Naira'] = df['Price_in_euros'] * 1600

X = df[['Ram', 'SSD', 'HDD', 'Company_enc', 'Prod_enc', 'ScreenRes_enc']]
y = df['Price_in_Naira']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)


def fuzzy_correct(user_input, valid_options, field_name='field', cutoff=70):
    match = process.extractOne(user_input, valid_options, score_cutoff=cutoff)
    if match:
        return match[0]
    else:
        raise ValueError(f"No close match found for {field_name}: '{user_input}'.")



window = Tk()
window.geometry("500x550")
window.configure(bg="#f0f4ff")
window.title('PC price Predicter')
window.resizable(width=False, height=False)

title_label = Label(window, text="💻 PC Price Predicter", font=('Calibri', 26, 'bold'), fg="#3b3bff", bg="#f0f4ff")
title_label.pack(pady=10)

def pred():
    try:
        ram = int(ram_entry.get())
        ssd = int(ssd_entry.get())
        hdd = int(hdd_entry.get())
        company_corrected = fuzzy_correct(comp_entry.get(), comp, 'Company')
        product_corrected = fuzzy_correct(prod_entry.get(), list(product_encoder.classes_), 'Product', cutoff=60)
        screenres_corrected = fuzzy_correct(res_entry.get(), res, 'ScreenResolution')

        company_encoded = company_encoder.transform([[company_corrected]])[0][0]
        screenres_encoded = screenres_encoder.transform([[screenres_corrected]])[0][0]
        product_encoded = product_encoder.transform([product_corrected])[0]

        features = [[ram, ssd, hdd, company_encoded, product_encoded, screenres_encoded]]
        prediction = model.predict(features)[0]
        pred_label.config(text=f"Estimated Price: ₦{int(prediction):,}", fg="green")

    except Exception as e:
        messagebox.showerror("Error", str(e))

da_frame = Frame(window, bg="#f0f4ff")
da_frame.pack(pady=10)

def design(label, row, options):
    lab1 = Label(da_frame, text=label, background='#f0f4ff')
    lab1.grid(row=row, column=0, pady=5, padx=10)
    com = ttk.Combobox(da_frame, value=options)
    com.current(0)
    com.grid(row=row, column=1, padx=10, pady=5)
    return com

ram_entry = design("RAM:", 0, mem)
ram_entry.current(1)
ssd_entry = design("SDD:", 1, disk)
ssd_entry.current(2)
hdd_entry = design("HDD:", 2, disk)
comp_entry = design("Company:", 3, comp)
prod_entry = design("Product:", 4, prod)
res_entry = design("Screen Resolution:", 5, res)

def on_company_change(event):
    selected_company = comp_entry.get()
    products = company_product_map.get(selected_company, [])
    prod_entry.config(state="readonly")
    prod_entry['values'] = products
    product_entry.current(0)
    if products:
        prod_entry.set(products[0])

comp_entry.bind('<<ComboboxSelected>>', on_company_change)




pred_btn = Button(window, text="Generate Price", command=pred, width=20, font=('Calibri', 12), bg='white', bd=0)
pred_btn.pack(pady=10)

pred_label = Label(window, text="Estimated Price: ₦0", font=('Calibri', 18), bg="#f0f4ff", fg="#444")
pred_label.pack(pady=20)

window.mainloop()